In [29]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire les détails d'un article à partir de son URL
def scrape_article(article_url):
    try:
        # Faire une requête HTTP pour obtenir le contenu de la page
        response = requests.get(article_url, timeout=10)  # Timeout de 10 secondes
        response.raise_for_status()  # Vérifier si la requête a réussi

        # Extraire le HTML de la page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraire le titre
        title_tag = soup.find("h1", class_="artical-content-heads")  # Modifier la balise si nécessaire
        title = title_tag.text.strip() if title_tag else "Titre non disponible"

        # Extraire le contenu
        content_tag = soup.find("div", class_="bodystr")  # Modifier la balise et la classe si nécessaire
        content = content_tag.text.strip() if content_tag else "Contenu non disponible"

        # Extraire la date
        date_tag = soup.find('span', class_='story_date')  # Modifier la balise et la classe si nécessaire
        date = date_tag.text.strip() if date_tag else "Date non disponible"

        return {
            'title': title,
            'content': content,
            'url': article_url,
            'date': date
        }
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du chargement de l'article {article_url}: {e}")
        return None  # Retourner None en cas d'erreur

# Fonction pour scraper une catégorie donnée
def scrape_category(base_url, category_name, max_articles=1250):
    all_articles = []
    page_number = 1

    # Boucle pour parcourir les pages jusqu'à atteindre max_articles
    while len(all_articles) < max_articles:
        # Construire l'URL de la page actuelle
        url = base_url.format(page_number)
        print(f"Scraping de la page {page_number} de la catégorie {category_name} : {url}")

        try:
            # Faire une requête HTTP pour obtenir le contenu de la page
            response = requests.get(url, timeout=10)  # Timeout de 10 secondes
            response.raise_for_status()  # Vérifier si la requête a réussi

            # Extraire le HTML de la page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire tous les liens des articles
            article_links = []
            articles = soup.find_all('div', class_='akhirul-akhbar-hero-sec')  # Modifier la classe si nécessaire
            for article in articles:
                link_tag = article.find('a', href=True)
                if link_tag:
                    article_url = link_tag['href']
                    if not article_url.startswith('http'):
                        article_url = 'https://www.akhbarona.com' + article_url  # Compléter l'URL si nécessaire
                    article_links.append(article_url)

            # Scraper chaque article
            for link in article_links:
                if len(all_articles) >= max_articles:
                    break
                print(f"Scraping de l'article : {link}")
                article_details = scrape_article(link)
                if article_details:  # Ignorer les articles qui retournent None
                    all_articles.append(article_details)

            # Passer à la page suivante
            page_number += 1

            # Vérifier s'il y a une page suivante
            next_page = soup.find('a', class_='pagi-active-1')  # Modifier la balise et la classe si nécessaire
            if not next_page:
                print(f"Fin de la pagination pour la catégorie {category_name}.")
                break

        except requests.exceptions.RequestException as e:
            print(f"Erreur lors du chargement de la page {url}: {e}")
            break  # Sortir de la boucle en cas d'erreur

    return all_articles

# URLs de base pour chaque catégorie
categories = {
    'politic': 'https://www.akhbarona.com/politic/index.{}.html',
    'economy': 'https://www.akhbarona.com/economy/index.{}.html',
   'world': 'https://www.akhbarona.com/world/index.1.html',
    'culture': 'https://www.akhbarona.com/culture/index.{}.html'
}

# Scraper chaque catégorie et sauvegarder les résultats dans des fichiers JSON distincts
for category_name, base_url in categories.items():
    articles = scrape_category(base_url, category_name)
    
    # Sauvegarder les articles dans un fichier JSON
    with open(f'{category_name}_articles.json', 'w', encoding='utf-8') as f:
        json.dump(articles, f, ensure_ascii=False, indent=4)
    
    print(f"{len(articles)} articles de la catégorie {category_name} extraits avec succès !")

Scraping de la page 1 de la catégorie politic : https://www.akhbarona.com/politic/index.1.html
Scraping de l'article : https://www.akhbarona.com/politic/401382.html
Scraping de l'article : https://www.akhbarona.com/politic/401380.html
Scraping de l'article : https://www.akhbarona.com/politic/401294.html
Scraping de l'article : https://www.akhbarona.com/politic/401290.html
Scraping de l'article : https://www.akhbarona.com/politic/401270.html
Scraping de l'article : https://www.akhbarona.com/politic/401210.html
Scraping de l'article : https://www.akhbarona.com/politic/401178.html
Scraping de l'article : https://www.akhbarona.com/politic/401139.html
Scraping de l'article : https://www.akhbarona.com/politic/401133.html
Scraping de l'article : https://www.akhbarona.com/politic/401132.html
Scraping de l'article : https://www.akhbarona.com/politic/401113.html
Scraping de l'article : https://www.akhbarona.com/politic/401111.html
Scraping de l'article : https://www.akhbarona.com/politic/401109.